In [ ]:
import faultier
import serial
import time

from tqdm.notebook import trange
ft = faultier.Faultier()
lp = faultier.LiveMarkerPlotOld()

ser = serial.Serial(ft.get_serial_path(), baudrate=115200)
ser.timeout = 0.2
ft.configure_glitcher(
    power_cycle_output = faultier.OUT_MUX0,
    power_cycle_length = 300000,
    trigger_source = faultier.TRIGGER_IN_EXT0,
    trigger_type = faultier.TRIGGER_PULSE_POSITIVE,
    glitch_output = faultier.OUT_CROWBAR
)

success = []
fails = []
detected = []
for d in trange(1, 500000):
    # for p in range(350, 370):
    for p in range(300, 370):
        if ser.in_waiting:
            ser.read(ser.in_waiting)
        ft.glitch(d, p)
        r = ser.read(3)
        try:
            if(r[2] == ord("X")):
                print(f"Successful glitch - Delay: {d} Pulse: {p}")
                success.append([d, p])
                lp.updatCe([success, fails, detected])
            elif(r[2] == ord("G")):
                print(f"Glitch detector triggered - Delay: {d} Pulse: {p}")
                detected.append([d, p])
                lp.update([success, fails, detected])
                # time.sleep(5)
            else:
                fails.append([d, p])
        except:
            fails.append([d, p])
            